In [1]:
import pandas as pd
import re

def extract_totals_from_details(text):
    # Define the regex pattern to capture totals from the text
    totals_pattern = re.compile(
        r"Total baggage\s*:\s*(?P<total_baggage>[\d\.]+)\s*KG\s*"
        r"Total cargo\s*:\s*(?P<total_cargo>[\d\.]+)\s*KG\s*"
        r"Total MNC\s*:\s*(?P<total_mnc>[\d\.]+)\s*KG\s*"
        r"Total mail\s*:\s*(?P<total_mail>[\d\.]+)\s*KG"
    )
    # Search for the pattern
    match = totals_pattern.search(text)
    return match.groupdict() if match else {"total_baggage": None, "total_cargo": None, "total_mnc": None, "total_mail": None}

# Load your CSV file
file_path = '/Users/nikyakovlev/Documents/GitHub/aircraft_load/data_engineering/niks_data_mata/UpdateLoadtableAction/UpdateLoadTableAction_entries_MNOP.csv'
df = pd.read_csv(file_path)

# Apply the function to extract totals from the 'entry_details' column
df_totals = df['entry_details'].apply(extract_totals_from_details)

# Convert the series of dictionaries to a DataFrame
baggage_data_df = pd.DataFrame(list(df_totals))

# Include the additional columns
df['timestamp'] = df['creation_time']
df['creation_time'] = pd.to_datetime(df['creation_time']).dt.to_period('M').astype(str)
df['creation_time'] = df['creation_time'].str.replace('-04', '-05')
df['combined'] = df['creation_time'] + '_' + df['airline_code'] + '_' + df['flight_number'].astype(str) + '_' + df['flight_date'].astype(str)

# Add the departure airport column to the baggage_data_df
baggage_data_df['departure_airport'] = df['departure_airport']

# Add the combined column and timestamp to the baggage_data_df
baggage_data_df['combined'] = df['combined']
baggage_data_df['timestamp'] = df['timestamp']

# Display the first few rows of the final DataFrame
print(baggage_data_df.head())

# Save the final DataFrame to a new CSV file
output_file_path = '/Users/nikyakovlev/Documents/GitHub/aircraft_load/data_engineering/niks_data_mata/UpdateLoadtableAction/extracted_totals_MNOP.csv'
baggage_data_df.to_csv(output_file_path, index=False)


  total_baggage total_cargo total_mnc total_mail departure_airport  \
0          None        None      None       None               DUB   
1       1820.00        0.00     88.00       0.00               DUB   
2          None        None      None       None               DUB   
3       1820.00        0.00    264.00       0.00               DUB   
4          None        None      None       None               DUB   

                combined            timestamp  
0  2024-05_MN_1446.0_1.0  2024-04-30 05:35:40  
1  2024-05_MN_1446.0_1.0  2024-04-30 05:35:40  
2  2024-05_MN_1446.0_1.0  2024-04-30 05:36:05  
3  2024-05_MN_1446.0_1.0  2024-04-30 05:36:05  
4  2024-05_MN_1446.0_1.0  2024-04-30 05:35:56  


In [2]:
import pandas as pd

# Load the CSV file
file_path = '/Users/nikyakovlev/Documents/GitHub/aircraft_load/data_engineering/niks_data_mata/UpdateLoadtableAction/extracted_totals_MNOP.csv'  # Update this path to your CSV file location
df = pd.read_csv(file_path)

# Ensure that timestamp is a datetime object for proper sorting
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Filter rows where all specified totals are not null
df_filtered = df.dropna(subset=['total_baggage', 'total_cargo', 'total_mnc', 'total_mail'])

# Sort by 'timestamp' and 'combined', then drop duplicates keeping the last (most recent) entry for each 'combined'
df_latest = df_filtered.sort_values(by=['timestamp', 'combined']).drop_duplicates('combined', keep='last')

# Save the final DataFrame to a new CSV file
output_file_path = '/Users/nikyakovlev/Documents/GitHub/aircraft_load/data_engineering/niks_data_mata/UpdateLoadtableAction/unique_extracted_totals_MNOP.csv'  # Update this path to where you want to save the new CSV
df_latest.to_csv(output_file_path, index=False)

# Display the first few rows of the final DataFrame to verify the output
print(df_latest.head())



       total_baggage  total_cargo  total_mnc  total_mail departure_airport  \
19750          884.0          0.0        0.0         0.0               DUB   
19763         1599.0          0.0        0.0         0.0               DUB   
19833          585.0        310.0        0.0       654.0               DUB   
19861         1027.0        898.0        0.0         0.0               DUB   
19950         1118.0          0.0        0.0         0.0               DUB   

                     combined           timestamp  
19750  2024-05_MN_1680.0_30.0 2024-04-30 04:43:25  
19763  2024-05_MN_1450.0_30.0 2024-04-30 04:51:19  
19833  2024-05_MN_1602.0_30.0 2024-04-30 04:53:25  
19861  2024-05_MN_1592.0_30.0 2024-04-30 04:58:24  
19950  2024-05_MN_1152.0_30.0 2024-04-30 05:07:28  
